In [0]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_csv("/content/drive/My Drive/data/hit_song/train_info.tsv", encoding='utf-8', delimiter='\t')
label = pd.read_csv("/content/drive/My Drive/data/hit_song/train_rank.csv")
track_info = pd.read_csv("/content/drive/My Drive/data/hit_song/all_track_info.csv")
track_audio = pd.read_csv("/content/drive/My Drive/data/hit_song/all_track_audio_features.csv")
test = pd.read_csv("/content/drive/My Drive/data/hit_song/test_info.tsv", encoding='utf-8', delimiter='\t')

In [4]:
train_set = train.merge(label, left_on='ID', right_on='ID')
train_set["dataset"] = 'train'
test["label"] = -1
test["dataset"] = 'test'
df = pd.concat([train_set, test])

df = df.merge(track_audio,  left_on='ID', right_on='ID')
df = df.merge(track_info, left_on='ID', right_on='ID', how='left')
df.head()

,ID,title,artist_name,artist_id,composers_name,composers_id,release_time,label,dataset,Unnamed: 0_x,lowlevel.average_loudness,lowlevel.barkbands_crest.mean,lowlevel.barkbands_crest.stdev,lowlevel.barkbands_flatness_db.mean,lowlevel.barkbands_flatness_db.stdev,lowlevel.barkbands_kurtosis.mean,lowlevel.barkbands_kurtosis.stdev,lowlevel.barkbands_skewness.mean,lowlevel.barkbands_skewness.stdev,lowlevel.barkbands_spread.mean,lowlevel.barkbands_spread.stdev,lowlevel.dissonance.mean,lowlevel.dissonance.stdev,lowlevel.dynamic_complexity,lowlevel.erbbands_crest.mean,lowlevel.erbbands_crest.stdev,lowlevel.erbbands_flatness_db.mean,lowlevel.erbbands_flatness_db.stdev,lowlevel.erbbands_kurtosis.mean,lowlevel.erbbands_kurtosis.stdev,lowlevel.erbbands_skewness.mean,lowlevel.erbbands_skewness.stdev,lowlevel.erbbands_spread.mean,lowlevel.erbbands_spread.stdev,lowlevel.hfc.mean,lowlevel.hfc.stdev,lowlevel.loudness_ebu128.integrated,lowlevel.loudness_ebu128.loudness_range,lowlevel.loudness_ebu128.momentary.mean,lowlevel.loudness_ebu128.momentary.stdev,...,rhythm.beats_loudness.stdev,rhythm.bpm,rhythm.bpm_histogram_first_peak_bpm,rhythm.bpm_histogram_first_peak_weight,rhythm.bpm_histogram_second_peak_bpm,rhythm.bpm_histogram_second_peak_spread,rhythm.bpm_histogram_second_peak_weight,rhythm.danceability,rhythm.onset_rate,tonal.chords_changes_rate,tonal.chords_number_rate,tonal.chords_strength.mean,tonal.chords_strength.stdev,tonal.hpcp_crest.mean,tonal.hpcp_crest.stdev,tonal.hpcp_entropy.mean,tonal.hpcp_entropy.stdev,tonal.key_edma.strength,tonal.key_krumhansl.strength,tonal.key_temperley.strength,tonal.tuning_diatonic_strength,tonal.tuning_equal_tempered_deviation,tonal.tuning_frequency,tonal.tuning_nontempered_energy_ratio,tonal.chords_key,tonal.chords_scale,tonal.key_edma.key,tonal.key_edma.scale,tonal.key_krumhansl.key,tonal.key_krumhansl.scale,tonal.key_temperley.key,tonal.key_temperley.scale,Unnamed: 0_y,album,genre,album_artist,track,lyric,length,copyright
0,1073748245,Đêm Chôn Dầu Vượt Biển,Như Quỳnh,551,Châu Đình An,5765,2017-10-01 22:07:00,7,train,4715,0.803644,11.713578,4.495951,0.178062,0.040354,3.206899,3.907893,0.502843,1.282988,12.635890,4.781546,0.439221,0.039377,3.381133,16.319588,7.274707,0.227460,0.053137,2.465001,3.769116,0.117311,0.726415,16.881458,10.196712,18.965084,14.671788,-18.571306,6.582901,-19.816732,3.657022,...,0.018779,122.142365,123.0,0.480000,110.0,0.545455,0.135135,0.780005,0.748981,0.039352,0.023148,0.485108,0.086305,19.135592,8.049803,1.339481,0.616099,0.753163,0.750701,0.747757,0.566135,0.007784,440.000000,0.601478,D,major,G,major,G,major,G,major,5321,30 Năm Viễn Xứ,Nhạc Trữ Tình,Various Artists,"(2, 14)",Đêm nay anh gánh dầu ra biển anh chôn \r\nAnh ...,363.334950,Thúy Nga
1,1073751978,Mùa Thu Trong Mưa,Minh Tuyết,455,Trường Sa,100105,2017-10-01 20:58:00,3,train,7917,0.972676,11.313860,5.094223,0.115021,0.046268,7.096139,10.796570,2.092896,1.439492,26.030527,16.313286,0.464209,0.021894,3.029083,9.717618,4.754533,0.121743,0.028430,0.144179,1.853869,0.472215,0.779543,51.735390,18.610188,21.840071,26.300795,-10.395282,1.879061,-10.776729,1.903993,...,0.018002,81.079300,81.0,0.280000,0.0,0.000000,0.000000,1.424859,3.744905,0.076389,0.013889,0.502246,0.102387,14.875016,7.787819,1.903883,0.782638,0.547513,0.559720,0.577588,0.596716,0.217171,434.193115,0.944516,C,minor,C,minor,C,minor,C,minor,8946,Xin Còn Gọi Tên Nhau-Tình khúc Trường Sa,Nhạc Trữ Tình,Various Artists,"(4, 18)",Chiều mưa không có em\r\nbờ đá công viên âm th...,243.235644,Thúy Nga
2,1073835561,Rồi Ánh Trăng Tan,Lưu Bích,450,Quốc Bảo,4355,2017-11-01 18:16:00,6,train,2372,0.941274,14.464849,5.344855,0.226093,0.093832,18.860538,31.805309,2.819379,2.629598,20.827469,20.129467,0.423720,0.049108,3.608222,14.645269,6.595244,0.194853,0.079173,3.116429,7.461062,0.921476,1.435056,41.564434,27.442163,30.379339,42.009789,-10.768950,3.072439,-11.192327,2.037850,...,0.063705,95.165367,96.0,0.379310,103.0,0.000000,0.033333,1.335825,3.744905,0.078704,0.016204,0.551

In [5]:
import numpy as np
import seaborn as sns
# Fill nan album
print("There is {} ratio is nan album".format(len(df[df["album"].isnull()])/len(df)))
df["album"]  = df["album"].fillna("")
df["len_album_name"] = df["album"].apply(lambda x: len(x.split(" ")))
df["isRemixAlbum"] = [ 1 if "Remix" in t else 0 for t in df["album"]]
df["isOSTAlbum"] = [ 1 if "OST" in t else 0 for t in df["album"]]
df["isSingleAlbum"] = [ 1 if "Single" in t else 0 for t in df["album"]]
df["isBeatAlbum"] = [ 1 if "Beat" in t else 0 for t in df["album"]]
df["isTopHitAlbum"] = [ 1 if "Top Hits" in t else 0 for t in df["album"]]
df["isCoverAlbum"] = [ 1 if "Cover" in t else 0 for t in df["album"]]
df["isEPAlbum"] = [ 1 if "EP" in t else 0 for t in df["album"]]
df["album_name_is_title_name"]= [1 if r.title in r.album  else 0 for i,r in df.iterrows() ]
df["album"] = df["album"].astype('category')
df["album"] =  df["album"].cat.codes

df["artist_name_cat"] = df["artist_name"].astype('category')
df["artist_name_cat"] =  df["artist_name_cat"].cat.codes
df["composers_name_cat"] = df["composers_name"].astype('category')
df["composers_name_cat"] =  df["composers_name_cat"].cat.codes
df["copyright_cat"] = df["copyright"].astype('category')
df["copyright_cat"] =  df["copyright_cat"].cat.codes

import re
def get_min_artist_id(s):
    ps = re.split(',|\.',s)
    ps = [int(p) for p in ps]
    return np.min(ps)

def get_max_artist_id(s):
    ps = re.split(',|\.',s)
    ps = [int(p) for p in ps]
    return np.max(ps)

df["artist_id_min"]=  df["artist_id"].apply(lambda x: get_min_artist_id(x))
df["artist_id_min_cat"] = df["artist_id_min"].astype('category')
df["artist_id_min_cat"] =  df["artist_id_min_cat"].cat.codes

df["composers_id_min"]=  df["composers_id"].apply(lambda x: get_min_artist_id(x))
df["composers_id_min_cat"] = df["composers_id_min"].astype('category')
df["composers_id_min_cat"] =  df["composers_id_min_cat"].cat.codes

df["artist_id_max"]=  df["artist_id"].apply(lambda x: get_max_artist_id(x))
df["artist_id_max_cat"] = df["artist_id_max"].astype('category')
df["artist_id_max_cat"] =  df["artist_id_max_cat"].cat.codes

df["composers_id_max"]=  df["composers_id"].apply(lambda x: get_max_artist_id(x))
df["composers_id_max_cat"] = df["composers_id_max"].astype('category')
df["composers_id_max_cat"] =  df["composers_id_max_cat"].cat.codes

# Fill genre
print("There is {} ratio is nan genre".format(len(df[df["genre"].isnull()])/len(df)))
df["genre"]  = df["genre"].fillna("No genre")
df["genre"] = df["genre"].astype('category')
df["genre"] =  df["genre"].cat.codes

# Fill album_artist
print("There is {} ratio is nan album_artist".format(len(df[df["album_artist"].isnull()])/len(df)))
df["album_artist"]  = df["album_artist"].fillna("No album_artist")
df["album_artist_contain_artistname"]= [1 if r.album_artist in r.artist_name  else 0 for i,r in df.iterrows() ]
df["album_artist"] = df["album_artist"].astype('category')
df["album_artist"] =  df["album_artist"].cat.codes

# Fill track
print("There is {} ratio is nan track".format(len(df[df["track"].isnull()])/len(df)))
df["track"]  = df["track"].fillna("(1,1)")
def tracknum_to_value(track_num):
    try:
        
        track_num = make_tuple(track_num)
        if track_num[0] is not None:
            return float(track_num[0]) / float(track_num[1])
        else:
            return 1.0
    except:
        return 1.0

df["track"] = df["track"].apply(lambda t: tracknum_to_value(t))


# Fill lyric
print("There is {} ratio is nan lyric".format(len(df[df["lyric"].isnull()])/len(df)))
df["lyric"]  = df["lyric"].fillna("")
df["islyric"] = df["lyric"].apply(lambda x: not len(x))



#--------------------------------------------------------
from dateutil import relativedelta
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from ast import literal_eval as make_tuple
df['no_artist'] = df.artist_name.apply(lambda x: len(x.split(",")))
df['no_composer'] = df.composers_name.apply(lambda x: len(x.split(",")))
df["freq_artist"] = df.groupby('artist_id')['artist_id'].transform('count')
df["freq_composer"] = df.groupby('composers_id')['composers_id'].transform('count')
df["datetime"] = pd.to_datetime(df.release_time)
df["year"] = df["datetime"].dt.year
df["month"] = df["datetime"].dt.month
df["hour"] = df["datetime"].dt.hour
df["day"] = df["datetime"].dt.day
df["weekday"] = df["datetime"].dt.weekday
df["len_of_songname"] = df["title"].apply(lambda x: len(x.split(" ")))
df["isRemix"] = [ 1 if "Remix" in t else 0 for t in df["title"]]
df["isOST"] = [ 1 if "OST" in t else 0 for t in df["title"]]
df["isBeat"] = [ 1 if "Beat" in t else 0 for t in df["title"]]
df["isVersion"] = [ 1 if "Version" in t else 0 for t in df["title"]]
df["isCover"] = [ 1 if "Cover" in t else 0 for t in df["title"]]


def find_num_song_release_in_final_month(df, day):
    month5th = day + relativedelta.relativedelta(months=5)
    month6th = day + relativedelta.relativedelta(months=6)  
    return len(df.datetime[(df.datetime >= month5th)&(df.datetime<=month6th)])



df["num_song_release_in_final_month"] = df.datetime.apply(lambda d:find_num_song_release_in_final_month(df ,d))


df_train = df[df.dataset=="train"]
df_test = df[df.dataset=="test"]

There is 0.23983899469860592 ratio is nan album
There is 0.0009817396426467701 ratio is nan genre
There is 0.23983899469860592 ratio is nan album_artist
There is 0.0 ratio is nan track
There is 0.6719026114274494 ratio is nan lyric


In [0]:
df['title_len'] = df['title'].apply(len)
df['art_len'] = df['artist_name'].apply(len)
df['comp_len'] = df['composers_name'].apply(len)

In [0]:
df['title'] = df.title.apply(lambda x: x.lower())
df['artist_name'] = df.artist_name.apply(lambda x: x.lower())
df['composers_name'] = df.composers_name.apply(lambda x: x.lower())

In [8]:
!pip install Unidecode
from unidecode import unidecode

     |████████████████████████████████| 245kB 5.0MB/s 


In [0]:
df['title'] = df.title.apply(lambda x: unidecode(x))
df['artist_name'] = df.artist_name.apply(lambda x: unidecode(x))
df['composers_name'] = df.composers_name.apply(lambda x: unidecode(x))

In [0]:
def number_of_letter(letter, name):
  num = 0
  for char in name:
    if char == letter:
      num+=1
  return num

In [0]:
char_lst='abcdefghijklmnopqrstuvwxyz'
for char in char_lst:
  df['no_of_'+char+'in_title'] = df.title.apply(lambda x: number_of_letter(char, x))
  df['no_of_'+char+'in_art'] = df.artist_name.apply(lambda x: number_of_letter(char, x))
  df['no_of_'+char+'in_comp'] = df.composers_name.apply(lambda x: number_of_letter(char, x))

In [0]:
df_train = df[df.dataset=="train"]
df_test = df[df.dataset=="test"]

In [15]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from math import sqrt

from sklearn.model_selection import train_test_split

# chosen_features = ["no_artist", "no_composer","freq_artist", "freq_composer","year", "month","hour", "day"]
# chosen_features = ["no_artist", "no_composer","freq_artist", "freq_composer","year", "month","hour", "day", "len_of_songname", 
#                    "isRemix", "isOST", "isBeat", "isVersion", "isCover",  "num_song_release_in_final_month",
#                   "length", "genre", "track","album_artist","album", "islyric", "album_artist_contain_artistname",
#                   "len_album_name", "isRemixAlbum", "isOSTAlbum", "isSingleAlbum", "album_name_is_title_name",
#                   "isBeatAlbum", "isCoverAlbum", "artist_name_cat","composers_name_cat","copyright_cat" ,
#                   "artist_id_min_cat", "composers_id_min_cat",  "artist_id_max_cat", "composers_id_max_cat"]
chosen_features = ['day', 'freq_artist', 'artist_id_min', 'month', 'genre', 'artist_name_cat',
                   'isBeat', 'artist_id_min_cat', 'num_song_release_in_final_month', 'artist_id_max',
                   'art_len', 'album_artist', 'album', 'isSingleAlbum', 'composers_id_min', 'length',
                   'freq_composer', 'no_of_nin_art', 'len_album_name', 'weekday',
                   'composers_id_max', 'artist_id_max_cat', 'title_len', 
                   'composers_name_cat', 'len_of_songname', 'hour', 'no_of_gin_art']


X = df_train[chosen_features]
y = df_train.label
df1 = X[X.isna().any(axis=1)]
print(df1)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

param = {
    'bagging_freq': 20,          
    'bagging_fraction': 0.95,   'boost_from_average':'false',   
    'boost': 'gbdt',             'feature_fraction': 0.6,     'learning_rate': 0.001,
    'max_depth': -1,             'metric':'root_mean_squared_error', 'min_data_in_leaf': 9,   
    'min_sum_hessian_in_leaf': 20.0,   'num_leaves': 100,            
    'num_threads': 8,              'tree_learner': 'serial',   'objective': 'regression',
    'reg_alpha': 0.1302650970728192, 'reg_lambda': 0.3603427518866501,'verbosity': 1
}

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=99999)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))
train_pred = np.zeros(len(df_train))
labels= df_train.label
for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values, df_train.label.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][chosen_features], label=labels.iloc[trn_idx])
    val_data = lgb.Dataset(df_train.iloc[val_idx][chosen_features], label=labels.iloc[val_idx])
    clf = lgb.train(param, trn_data, 1000000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 20000)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][chosen_features], num_iteration=clf.best_iteration)
    predictions += clf.predict(df_test[chosen_features], num_iteration=clf.best_iteration) / folds.n_splits
    val_pred += clf.predict(X_valid[chosen_features], num_iteration=clf.best_iteration) / folds.n_splits

Empty DataFrame
Columns: [day, freq_artist, artist_id_min, month, genre, artist_name_cat, isBeat, artist_id_min_cat, num_song_release_in_final_month, artist_id_max, art_len, album_artist, album, isSingleAlbum, composers_id_min, length, freq_composer, no_of_nin_art, len_album_name, weekday, composers_id_max, artist_id_max_cat, title_len, composers_name_cat, len_of_songname, hour, no_of_gin_art]
Index: []
Fold 0
Training until validation scores don't improve for 20000 rounds.
[5000]	training's rmse: 1.21225	valid_1's rmse: 1.75253
[10000]	training's rmse: 0.855697	valid_1's rmse: 1.68791
[15000]	training's rmse: 0.65646	valid_1's rmse: 1.67181
[20000]	training's rmse: 0.523041	valid_1's rmse: 1.6658
[25000]	training's rmse: 0.427372	valid_1's rmse: 1.66399
[30000]	training's rmse: 0.356745	valid_1's rmse: 1.66387
[35000]	training's rmse: 0.30341	valid_1's rmse: 1.66432
[40000]	training's rmse: 0.262377	valid_1's rmse: 1.66496
[45000]	training's rmse: 0.230182	valid_1's rmse: 1.66556
Earl

NameError: ignored

In [0]:
print("RMSE: {:<8.5f}".format(sqrt(mean_squared_error(df_train.label, oof))))

sub = pd.DataFrame({"ID": df_test.ID.values})
sub["label_lgbm"] = predictions
sub.to_csv("submission_1.csv", index=False)

In [1]:
r = {}
for yt, yp  in zip(y_valid, pre):
    if yt in sorted(r.keys()):
        r[yt].append(yp)
    else:
        r[yt] = [yp]

x1 = sorted(r.keys())
values = r.values()
means= [np.mean(value) for value in values]
mi= [np.min(value) for value in values]
ma= [np.max(value) for value in values]
quan25= [np.quantile(value, 0.1) for value in values]
quan75= [np.quantile(value, 0.9) for value in values]

fig = plt.figure(figsize=(10,7))
plt.plot(x1, means, label="Mean Rank prediction")
plt.fill_between(x1, mi,ma,color='b', alpha=0.2 , label="Min-Max Rank prediction")
plt.fill_between(x1, quan25,quan75,color='r', alpha=0.5 ,label="10-th to 90-th Rank prediction")
plt.legend(loc='upper left')
plt.xlabel("Actual Rank")
plt.ylabel("Predicted Rank")

    

NameError: name 'y_valid' is not defined

In [0]:
print("RMSE: {:<8.5f}".format(sqrt(mean_squared_error(df_train.label, oof))))

new_data = pd.DataFrame({"ID": df_train.ID.values})
new_data["label_lgbm"] = train_pred

In [0]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(30,30))
lgb.plot_importance(clf, max_num_features=30,importance_type='gain')

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

clf2 = LinearRegression()
clf2.fit(X_train, y_train)
pred_train2 = clf2.predict(X_train)
pred_val2 = clf2.predict(X_valid)
preds2 = clf2.predict(df_test[chosen_features])
print('rmse_train: '+ str(np.sqrt(mse(pred_train2, y_train))))
print('rmse_val: '+ str(np.sqrt(mse(pred_val2, y_valid))))

new_data['label_lr'] = pred_val2
sub['label_lr'] = preds2

In [0]:
from sklearn.ensemble import RandomForestRegressor

clf3 = RandomForestRegressor(n_estimators=1000, max_depth=6, max_features="auto", random_state=123)
clf3.fit(X_train,y_train)
pred_train3 = clf3.predict(X_train)
pred_val3 = clf3.predict(X_valid)
pred3 = clf3.predict(X)
preds3 = clf3.predict(df)
print('rmse_train: '+ str(np.sqrt(mse(pred_train3))))
print('rmse_val: '+ str(np.sqrt(mse(pred_val3))))

new_data['label_rf'] = pred3
sub['label_rf'] = preds3


In [0]:
new_data.to_csv("new_data.csv", index=False)
sub.to_csv("submission_1.csv", index=False)

In [52]:
new_data.head()

,ID,label_lgbm,label_lr,label_rf
0,1073748245,6.011089,4.568815,4.677053
1,1073751978,3.651503,4.725702,4.543363
2,1073835561,5.959038,4.423996,5.019870
3,1073856553,2.771326,4.204129,4.693034
4,1073929630,6.720977,4.693176,5.430007


In [0]:
new_data.drop(columns=['ID'], inplace=True)
X_train, X_valid, y_train, y_valid = train_test_split(new_data, y, test_size=0.2, random_state=42)

In [56]:
from sklearn.ensemble import RandomForestRegressor

clft = RandomForestRegressor(n_estimators=1000, max_depth=6, max_features="auto", random_state=123)
clft.fit(X_train,y_train)

pred_train3 = clft.predict(X_train)
pred_val3 = clft.predict(X_valid)
print('rmse_train: '+ str(np.sqrt(mse(pred_train3, y_train))))
print('rmse_val: '+ str(np.sqrt(mse(pred_val3, y_valid))))

rmse_train: 0.43858874067310083
rmse_val: 0.48413638607377424


In [0]:
final_pred = clft.predict(sub)

In [0]:
new_sub = pd.DataFrame({"ID": df_test.ID.values})
new_sub["label"] = final_pred
new_sub.to_csv('sub.csv', index=False, header=False)